In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle as pkl
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils import resample
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import time
from alibi.explainers import CounterFactual

In [8]:
print('Eager execution enabled: ', tf.executing_eagerly()) # False
tf.compat.v1.disable_eager_execution()
# print('Eager execution enabled: ', tf.executing_eagerly()) # False

Eager execution enabled:  True


In [3]:
FOLDERPATH = '../'

In [4]:
modelname = 'baseline-model.h5'
model = load_model(FOLDERPATH+'models/'+modelname)
data_path = FOLDERPATH+'/codes/data/'
train_data = pd.read_csv(data_path+'v3.2.2_train.csv')
test_data = pd.read_csv(data_path+'test.csv')

In [5]:
y_train,y_test = train_data[['TOT_fast','TOT_med_fast','TOT_med','TOT_med_slow','TOT_slow']], test_data[['TOT_fast','TOT_med_fast','TOT_med','TOT_med_slow','TOT_slow']]
X_train,X_test = train_data.drop(['Unnamed: 0','TOT_fast','TOT_med_fast','TOT_med','TOT_med_slow','TOT_slow'],axis=1),test_data.drop(['Unnamed: 0','TOT_fast','TOT_med_fast','TOT_med','TOT_med_slow','TOT_slow'],axis=1)

In [6]:
shape = (1,) + X_train.shape[1:]
target_proba = 1.0
tol = 0.01 # want counterfactuals with p(class)>0.99
target_class = 'other' # any class other than 7 will do
max_iter = 1000
lam_init = 1e-1
max_lam_steps = 10
learning_rate_init = 0.1
feature_range = (X_train.min(),X_train.max())

In [9]:

# initialize explainer
cf = CounterFactual(model, shape=shape, target_proba=target_proba, tol=tol,
                    target_class=target_class, max_iter=max_iter, lam_init=lam_init,
                    max_lam_steps=max_lam_steps, learning_rate_init=learning_rate_init,
                    feature_range=feature_range)

start_time = time()
explanation = cf.explain(X_test)
print('Explanation took {:.3f} sec'.format(time() - start_time))

ValueError: Calling `Model.predict` in graph mode is not supported when the `Model` instance was constructed with eager mode enabled. Please construct your `Model` instance in graph mode or call `Model.predict` with eager mode enabled.